https://python.langchain.com/docs/tutorials/rag/
</br>https://python.langchain.com/docs/integrations/text_embedding/
</br>https://platform.openai.com/settings/organization/billing/overview

In [ ]:
import json

from pathlib import Path

import pandas as pd
import pypdf as pdf

from langchain_core.documents import Document
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_core.prompts.chat import PromptTemplate
from langchain_openai import ChatOpenAI

from settings import OPENAI_API_KEY

In [2]:
PROMPT_TEMPLATE = Path('/Users/caiopavesi/Code/bmw-job-matcher/data/prompt.jinja').read_text()
APPLICANT_DOCUMENTS_FILES_PATH = Path('/Users/caiopavesi/Library/Mobile Documents/com~apple~CloudDocs/0/Work/Job applications/Templates/Latest')

In [3]:
loader = DirectoryLoader(APPLICANT_DOCUMENTS_FILES_PATH, glob = "*.pdf", loader_cls = PyPDFLoader)
docs_content = loader.load()

In [6]:
llm = ChatOpenAI(
    model = "gpt-4.1",
    temperature = 0,
    api_key = OPENAI_API_KEY
)

In [12]:
job_descriptions = pd.read_excel('../data/bmw_jobs.xlsx')

In [19]:
index = 56
job_description = job_descriptions.loc[index, ('description')]

In [20]:
prompt = PromptTemplate(
    template = PROMPT_TEMPLATE,
    template_format = "jinja2",
    input_variables = ["context", "question"],
)

In [21]:
prompt = prompt.invoke({"context": docs_content, "question": job_description})

In [22]:
answer = llm.invoke(prompt)

In [23]:
rate = json.loads(answer.content)

In [24]:
rate

{'score': 91.5,
 'strengths': "The candidate has a strong technical background in production and systems engineering, with direct and recent experience at BMW Group in multiple roles, including software-related tasks (Python, Power Platform, Excel/VBA, SQL). He has demonstrated hands-on project management, leadership, and cross-functional teamwork, as well as strong communication skills, as evidenced by multiple recommendation letters. His education is highly relevant, and he is in the advanced stages of his bachelor's degree, meeting the internship requirements. He also has international exposure and a clear motivation for the automotive sector and BMW specifically.",
 'weaknesses': 'Limited direct experience with C or C++ (though Python is well covered). German language skills are at a beginner level (A1), which may be a minor limitation for integration in some teams, though not a strict requirement. No explicit mention of experience in automotive series software development, but str

In [ ]:
Application = Document | list[Document]

def evalueate_applicantion(llm: ChatOpenAI, applicant: Application, job_description: str) -> dict[str, float | str | bool]:
    """
    Evaluate an applicant's documents against a job description using a language model.
    Args:
        llm (ChatOpenAI): The language model to use for evaluation.
        applicant (Application): The applicant's documents, either a single Document or a list of Documents.
        job_description (str): The job description to evaluate against.
    Returns:
        dict[str, float | str | bool]: A dictionary containing the evaluation results, including a score and a recommendation.
    """

    prompt = PromptTemplate(
        template = PROMPT_TEMPLATE,
        template_format = "jinja2",
        input_variables = ["context", "question"],
    )

    prompt = prompt.invoke({"context": applicant, "question": job_description})

    answer = llm.invoke(prompt)

    return json.loads(answer.content)